In [17]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [20]:
def generate_sentence(max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(max_len / 3))
        return n*"a" + n*"b" + n*"c"
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(0, max_len - n_0 - n_1 + 1)
        return n_0*"a" + n_1*"b" + n_2*"c"

def create_data(size=10000, balance=0.5):
    data = []
    for i in range(int(size*balance)):
        data.append((generate_sentence(pos=True), 1))
    for i in range(int((size - (size*balance)))):
        data.append((generate_sentence(pos=False), 0))
    
    random.shuffle(data)
    return data

data = create_data()
print(data)

[('abc', 1), ('', 1), ('aaaaaaabbccccccccccc', 0), ('aaaaabbbbbccccc', 1), ('aaaaabbbbbbbbbbbbbbb', 0), ('aaaaabbbbbccccc', 1), ('aaaabbbbcccc', 1), ('aaaaabbbccc', 0), ('abc', 1), ('bbbbbcccc', 0), ('aaaabbbbcccc', 1), ('aab', 0), ('aaaaaaaaaabbccccccc', 0), ('aaabbbbbbbbbbbbbb', 0), ('', 1), ('aaaaaaaaaaaaaaaabbbb', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaaaaaaaab', 0), ('abbbbbbbbbbbbbbbbbbb', 0), ('aaaaaaaaaabbbbccccc', 0), ('aaabbbccc', 1), ('abc', 1), ('aaaabbbbcccc', 1), ('aaaaaaaaaaaaaabbbbb', 0), ('abc', 1), ('aabbcc', 1), ('aaaaaabbbbbbcccccc', 1), ('aabbcc', 1), ('', 1), ('aaaaaaabcccc', 0), ('aaaaaaaaabbbbbbbbbbc', 0), ('aaaabbbbcccc', 1), ('aaaaaaaaaabbbbbb', 0), ('aaaaaaaaaabbbbbbbbbb', 0), ('aaaaabbbbbcccc', 0), ('aaaaaaaaaccccccccccc', 0), ('', 1), ('aaaaaaabbbbbbbbbc', 0), ('aaaaaaaaaaaaaaaaabc', 0), ('aaaaaaaaaaaaaaaabbbb', 0), ('aaaaabbbccccccccc', 0), ('bbbbbbbbbbbbbc', 0), ('', 1), ('aaaaaabcc', 0), ('bbbbbbbbbbbbbbbbc', 0), ('aaaaaaaaaaaaaabb', 0), ('aabbcc', 1), ('aa